In [ ]:
#pip install -r ../requirements.txt

In [ ]:
import os
import sys


try:
    os.chdir("../")  
    print("Directorio de trabajo cambiado a:", os.getcwd())
except FileNotFoundError:
    print("No se pudo cambiar el directorio, verifica la ruta.")

project_root = os.path.abspath(os.path.join(os.getcwd()))
if project_root not in sys.path:
    sys.path.append(project_root)
    
print("sys.path actualizado. Directorio base:", project_root)


In [ ]:
import os
import time
import logging
import requests
import pandas as pd
from tqdm import tqdm

# ============ CONFIGURACIÓN ============
MUSICBRAINZ_ENDPOINT = "https://musicbrainz.org/ws/2/artist/"
HEADERS = {
    "User-Agent": "workshop2/1.0 (ejemplo@gmail.com)"
}

# Ruta para 'artists.csv'
ARTISTS_CSV = ('data/artists.csv')
RETRY_LIMIT = 3  # Límite de reintentos en caso de error
BATCH_SIZE = 50  # Tamaño del lote para consultas
RESULTS_PER_PAGE = 100  # Ajuste el número de resultados por página (ajústelo según la capacidad de la API)

# Configuración del logging para mostrar el proceso de ejecución
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# ============ FUNCIONES ============

def limpiar_nombre(nombre: str) -> str:
    """Limpia los nombres de los artistas eliminando caracteres no deseados."""
    if pd.isna(nombre) or not nombre.strip():
        return None
    nombre = nombre.replace("\\", "")
    nombre = nombre.replace('"', '')
    nombre = nombre.replace("'", "")
    nombre = nombre.replace("/", " ")
    nombre = nombre.replace("&", "and")
    nombre = nombre.strip()
    return nombre

def _cargar_y_limpiar_artistas(ruta_csv: str) -> list:
    """Carga el CSV, limpia y extrae los artistas únicos."""
    df = pd.read_csv(ruta_csv, header=None, names=["raw"])
    nombres_limpios = [limpiar_nombre(nombre) for nombre in df["raw"]]
    artistas_unicos = sorted(set([nombre for nombre in nombres_limpios if nombre]))
    logging.info(f"✅ Total artistas únicos: {len(artistas_unicos)}")
    return artistas_unicos

def _consultar_musicbrainz(artistas_unicos: list) -> list:
    """Consulta la API de MusicBrainz para obtener información de los artistas en lotes y manejar la paginación."""
    resultados = []
    logging.info("🎧 Consultando MusicBrainz...")

    with tqdm(total=len(artistas_unicos), desc="🔎 MusicBrainz") as pbar:
        for i in range(0, len(artistas_unicos), BATCH_SIZE):
            lote_artistas = artistas_unicos[i:i + BATCH_SIZE]
            query = ' OR '.join([f'artist:"{artista}"' for artista in lote_artistas])
            page = 1
            while True:
                info_lote = _buscar_artista_musicbrainz_lote(query, page)

                if info_lote:
                    resultados.extend(info_lote)
                else:
                    for artista in lote_artistas:
                        logging.warning(f"⚠  No se encontró información para: {artista}")

                # Si hay más páginas, continuar con la siguiente página
                if len(info_lote) < RESULTS_PER_PAGE:
                    break  # No hay más resultados en esta página

                page += 1  # Incrementar la página para obtener más resultados
                time.sleep(1.1)  # Para evitar rate limit de la API
                pbar.update(len(lote_artistas))

    return resultados

def _buscar_artista_musicbrainz_lote(query: str, page: int) -> list:
    """Realiza la consulta a MusicBrainz para un lote de artistas con paginación."""
    params = {
        "query": query,
        "fmt": "json",
        "offset": (page - 1) * RESULTS_PER_PAGE,  # Desplazamiento para la página
        "limit": RESULTS_PER_PAGE  # Número de resultados por página
    }

    # Intentar la consulta hasta un máximo de RETRY_LIMIT veces en caso de error
    for intento in range(RETRY_LIMIT):
        try:
            logging.info(f"🔄 Intentando consulta para el lote (Intento {intento + 1}/{RETRY_LIMIT}) - Página {page}")
            response = requests.get(MUSICBRAINZ_ENDPOINT, headers=HEADERS, params=params, timeout=10)
            response.raise_for_status()  # Verifica si la respuesta fue exitosa (código 200)
            data = response.json()

            # Si se encuentra información de los artistas, extraerla
            if data["artists"]:
                resultados = []
                for artista in data["artists"]:
                    resultados.append({
                        "artist": artista.get("name", ""),
                        "country": artista.get("country", ""),
                        "type": artista.get("type", ""),
                        "disambiguation": artista.get("disambiguation", ""),
                        "life_begin": artista.get("life-span", {}).get("begin", ""),
                        "life_end": artista.get("life-span", {}).get("end", "")
                    })
                return resultados
            return None
        except requests.exceptions.RequestException as e:
            logging.error(f"❌ Error al consultar lote: {e}")
            time.sleep(2 * (intento + 1))  # Espera exponencial antes de reintentar
        except Exception as e:
            logging.error(f"❌ Error inesperado al consultar lote: {e}")
            return None

    logging.error(f"❌ Falló definitivamente la consulta del lote tras {RETRY_LIMIT} intentos.")
    return None

def extract_musicbrainz() -> pd.DataFrame:
    """Función principal que extrae los datos de MusicBrainz y los devuelve como un DataFrame."""
    # Cargar y limpiar los artistas
    artistas_unicos = _cargar_y_limpiar_artistas(ARTISTS_CSV)
    # Consultar MusicBrainz para obtener la información de los artistas
    resultados = _consultar_musicbrainz(artistas_unicos)
    # Definir las columnas que tendrá el DataFrame resultante
    columnas = ["artist", "country", "type", "disambiguation", "life_begin", "life_end"]

    # Imprimir los resultados
    logging.info("✅ Consulta completada. Resultados obtenidos:")
    for resultado in resultados:
        logging.info(f"Artista: {resultado['artist']}, País: {resultado['country']}, Tipo: {resultado['type']}")

    # Devolver los resultados como un DataFrame
    df_resultado = pd.DataFrame(resultados, columns=columnas)

    # Mostrar el DataFrame resultante
    return df_resultado


In [ ]:

# Llamar a la función principal para extraer los datos de MusicBrainz y obtener el DataFrame
df_resultado = extract_musicbrainz()

# Imprimir el DataFrame completo
print("\nDatos obtenidos de MusicBrainz:")
print(df_resultado)


In [ ]:
# Guardar el DataFrame en un archivo CSV llamado 'musicbrainz_artist.csv'
df_resultado.to_csv('musicbrainz_artist.csv', index=False)

# Confirmar que el archivo fue guardado
print("\nEl DataFrame ha sido guardado en 'musicbrainz_artist.csv'")
